In [ ]:
%pip install requests pandas

In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta # Importar datetime y timedelta
import json # Para un mejor print de los JSON de error o para depuración
import os # Importar os para trabajar con rutas de archivos
from concurrent.futures import ThreadPoolExecutor, as_completed # Importar para paralelismo

In [3]:
# --- PASO 1: OBTENER CONFIGURACIÓN INICIAL (INCLUYE TOKEN PARA SIGUIENTE PASO) ---
api_url_step1 = "https://127.0.0.1:5000/api/ConfigCompany/?opc=1"
loginConfig = None # Almacenará la respuesta JSON de la primera llamada
token_autenticacion_principal = None # Token extraído de loginConfig para la segunda llamada

print(f"--- Iniciando primera petición API para Configuración ---")
print(f"Intentando conectar a: {api_url_step1}")

try:
    # Realizar la solicitud GET
    response_step1 = requests.get(api_url_step1, verify=False, timeout=10) # timeout de 10 segundos

    print(f"Código de estado (Configuración): {response_step1.status_code}")

    if response_step1.status_code == 200:
        print("Conexión exitosa (Configuración).")
        try:
            loginConfig = response_step1.json()
            # print("Datos de configuración recibidos (JSON):") # Descomentar para depuración
            # print(json.dumps(loginConfig, indent=4)) 
            if isinstance(loginConfig, list) and len(loginConfig) > 0 and isinstance(loginConfig[0], dict) and 'id' in loginConfig[0]:
                token_autenticacion_principal = loginConfig[0]['id']
                print(f"Token de autenticación principal obtenido: {str(token_autenticacion_principal)[:15]}...")
            else:
                print("Respuesta JSON de configuración no tiene el formato esperado o 'id' no encontrado en el primer elemento.")
                print(f"Datos recibidos: {loginConfig}")
        except requests.exceptions.JSONDecodeError:
            print("No se pudo decodificar la respuesta de configuración como JSON.")
            print(f"Contenido (texto): {response_step1.text}")
    # Manejo de otros códigos de estado
    elif response_step1.status_code == 404:
        print("Error (Configuración - 404): Recurso no encontrado. Verifica la URL.")
    elif response_step1.status_code == 500:
        print("Error (Configuración - 500): Error interno del servidor.")
    else:
        print(f"Error en la solicitud de configuración: Código {response_step1.status_code}")
        print(f"Contenido: {response_step1.text}")

except requests.exceptions.ConnectionError as e:
    print(f"Error de conexión (Configuración): {e}. Asegúrate que el servidor API esté corriendo.")
except requests.exceptions.Timeout:
    print("Error (Configuración): La solicitud excedió el tiempo de espera.")
except requests.exceptions.RequestException as e:
    print(f"Ocurrió un error (Configuración): {e}")

--- Iniciando primera petición API para Configuración ---
Intentando conectar a: https://127.0.0.1:5000/api/ConfigCompany/?opc=1


C:\Users\geren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Código de estado (Configuración): 200
Conexión exitosa (Configuración).
Token de autenticación principal obtenido: j6oJTuCDcsbhbjy...


In [4]:
# --- PASO 2: OBTENER LISTA DE LICENCIAS/TIENDAS ---
print("\n--- Iniciando segunda petición API para Licencias ---")
df_licencias = pd.DataFrame() # Inicializar DataFrame vacío

if token_autenticacion_principal:
    api_url_licencia = "https://127.0.0.1:5000/api/LicenseV2/Boffice/"
    print(f"Intentando conectar a (Licencias): {api_url_licencia}")
    print(f"Usando token principal (ID como usuario para Basic Auth): {str(token_autenticacion_principal)[:15]}...")

    try:
        response_licencia = requests.get(
            api_url_licencia,
            auth=(str(token_autenticacion_principal), ''), # Asegurar que el token es string
            verify=False,
            timeout=15
        )
        print(f"Código de estado (Licencias): {response_licencia.status_code}")

        if response_licencia.status_code == 200:
            print("Conexión exitosa al endpoint de licencias.")
            try:
                datos_licencias = response_licencia.json()
                if isinstance(datos_licencias, list):
                    df_licencias = pd.DataFrame(datos_licencias)
                elif isinstance(datos_licencias, dict): # Si es un solo objeto
                    df_licencias = pd.DataFrame([datos_licencias])
                else:
                    print("El formato de los datos JSON de licencias no es una lista ni un diccionario simple.")
                
                if not df_licencias.empty:
                    print("\nDataFrame de Licencias creado exitosamente.")
                    if 'name' in df_licencias.columns:
                        licencias_originales_count = len(df_licencias)
                        df_licencias = df_licencias[~df_licencias['name'].astype(str).str.contains("INACTIVA", case=False, na=False)].copy()
                        licencias_filtradas_count = len(df_licencias)
                        if licencias_filtradas_count < licencias_originales_count:
                             print(f"Filtradas {licencias_originales_count - licencias_filtradas_count} licencias inactivas.")

                else:
                    print("El DataFrame de licencias está vacío (JSON podría ser lista vacía o no compatible).")
            except requests.exceptions.JSONDecodeError:
                print("No se pudo decodificar la respuesta de licencias como JSON.")
                print(f"Contenido (texto): {response_licencia.text}")
            except Exception as e:
                print(f"Error al procesar JSON de licencias o crear DataFrame: {e}")
        elif response_licencia.status_code == 401:
            print("Error de autenticación (Licencias - 401): No autorizado.")
        else:
            print(f"Error en la solicitud de licencias: Código {response_licencia.status_code}")
            print(f"Contenido (Licencias): {response_licencia.text}")
    except requests.exceptions.ConnectionError as e:
        print(f"Error de conexión con API de licencias: {e}")
    except requests.exceptions.Timeout:
        print("Error: La solicitud de licencias excedió el tiempo de espera.")
    except requests.exceptions.RequestException as e:
        print(f"Ocurrió un error en la solicitud de licencias: {e}")
else:
    print("No se obtuvo el token de autenticación principal. No se realizará la petición para licencias.")

if not df_licencias.empty:
    print("\n--- Resumen del DataFrame de Licencias (Activas) ---")
    print(df_licencias.head())
    print(f"Dimensiones: {df_licencias.shape} (tiendas activas a procesar)")
else:
    print("\nEl DataFrame de licencias está vacío o no se pudo generar.")


--- Iniciando segunda petición API para Licencias ---
Intentando conectar a (Licencias): https://127.0.0.1:5000/api/LicenseV2/Boffice/
Usando token principal (ID como usuario para Basic Auth): j6oJTuCDcsbhbjy...


C:\Users\geren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Código de estado (Licencias): 200
Conexión exitosa al endpoint de licencias.

DataFrame de Licencias creado exitosamente.
Filtradas 63 licencias inactivas.

--- Resumen del DataFrame de Licencias (Activas) ---
      id                                 name  \
1  00001                     [ BCO ] - BOCONÓ   
2  00002                [ TRU ] - VALERA SHOP   
3  00003  [ SMT ] - SABANA MENDOZA - TRUJILLO   
5  00005                 [ PFO ] - PUNTO FIJO   
6  00006             [ AVM ] - AVENIDA MARIÑO   

                                               token  
1  kdgSmPMvsRXEZ!0vjzNkPxNqHQjkKJok36iV3sZHQWV3KE...  
2  v4dZJyj/if7W8JQweG87wJma8Zztuj27J3PAUn5Vjoa/Gr...  
3  Lll49WWqkCnsZAxLf8HJUG7gY283/3/Ii2q8QCWxYpemYF...  
5  Pkcc!TobLVx5deTNZQGHUh2QVI1ZzX3zwyDoRu5zP8OYnL...  
6  los5A4YRyR4xIAFq66wGIVdpmoc/x/irwHegrkM8DUMdRy...  
Dimensiones: (191, 3) (tiendas activas a procesar)


In [9]:
# --- PASO 3: OBTENER CIERRES DE CAJA POR TIENDA (IMPLEMENTACIÓN CON THREADING) ---
print("\n--- Iniciando tercera petición API para Cierres de Caja por Tienda (en paralelo) ---")

# Función trabajadora que se ejecutará en cada hilo
def obtener_cierre_tienda(tienda, fecha_str):
    """
    Función de trabajo (worker) para un hilo. Obtiene el cierre de caja para una sola tienda.
    Retorna una tupla: ('success', DataFrame) o ('error', dict_de_error).
    """
    id_tienda_actual = tienda['id']
    tienda_token_especifico = tienda['token']
    tienda_name = tienda.get('name', str(id_tienda_actual))

    print(f"-> Iniciando petición para tienda: {tienda_name} (ID: {id_tienda_actual})")

    api_url_cierre_caja = f"https://127.0.0.1:5000/api/CashBox/Consolidated/{fecha_str}?bOffice={id_tienda_actual}&isExpress=false&corrName=&isOffline=false"

    try:
        response_cierre = requests.get(
            api_url_cierre_caja,
            auth=(str(tienda_token_especifico), ''),
            verify=False,
            timeout=30
        )

        if response_cierre.status_code == 200:
            datos_cierre = response_cierre.json()
            if isinstance(datos_cierre, dict) and datos_cierre.get('error') == 1 and "Index was outside the bounds of the array." in datos_cierre.get('message', ''):
                error_msg = datos_cierre.get('message')
                print(f"<- Error API específico para tienda {id_tienda_actual}: {error_msg}")
                return ('error', {'store_id': id_tienda_actual, 'store_name': tienda_name, 'error_message': error_msg, 'fecha_consulta': fecha_str})
            
            elif isinstance(datos_cierre, list) or (isinstance(datos_cierre, dict) and datos_cierre.get('error') != 1):
                df_cierre_tienda = pd.DataFrame(datos_cierre if isinstance(datos_cierre, list) else [datos_cierre])
                if not df_cierre_tienda.empty:
                    df_cierre_tienda['store_id'] = id_tienda_actual
                    df_cierre_tienda['store_name'] = tienda_name
                    print(f"<- Datos recibidos para tienda {id_tienda_actual} ({df_cierre_tienda.shape[0]} filas).")
                    return ('success', df_cierre_tienda)
                else:
                    print(f"<- Respuesta vacía o no procesable para tienda {id_tienda_actual}.")
                    return ('no_data', None)
            else:
                print(f"<- Respuesta JSON inesperada para tienda {id_tienda_actual}: {str(datos_cierre)[:200]}")
                return ('no_data', None)

        else: # Si el código de estado no es 200
            print(f"<- Error HTTP {response_cierre.status_code} para tienda {id_tienda_actual}.")
            return ('no_data', None)

    except requests.exceptions.RequestException as e:
        print(f"<- Excepción de red para tienda {id_tienda_actual}: {e}")
        return ('no_data', None)
    except json.JSONDecodeError:
        print(f"<- Error de decodificación JSON para tienda {id_tienda_actual}.")
        return ('no_data', None)
    except Exception as e:
        print(f"<- Excepción inesperada en worker para tienda {id_tienda_actual}: {e}")
        return ('no_data', None)


--- Iniciando tercera petición API para Cierres de Caja por Tienda (en paralelo) ---


In [20]:
# Listas para almacenar los resultados de los hilos
lista_df_cierres_caja = []
lista_errores_especificos_api = []
df_cierres_consolidado = pd.DataFrame() 

# MAX_WORKERS controla cuántas peticiones se hacen en paralelo. Ajusta según tu necesidad y la capacidad de la API.
MAX_WORKERS = 32

if not df_licencias.empty and 'id' in df_licencias.columns and 'token' in df_licencias.columns:
    fecha_ayer = datetime.now() - timedelta(days=1)
    fecha_ayer_str = '2025-05-02'#fecha_ayer.strftime("%Y-%m-%d")
    print(f"Obteniendo datos de cierre de caja para la fecha: {fecha_ayer_str}")

    # Usar ThreadPoolExecutor para paralelizar las peticiones
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor: # max_workers limita la concurrencia
        # Enviar todas las tareas al pool de hilos
        future_to_tienda = {executor.submit(obtener_cierre_tienda, tienda, fecha_ayer_str): tienda for index, tienda in df_licencias.iterrows()}
        
        # Recolectar resultados a medida que se completan
        for future in as_completed(future_to_tienda):
            tienda = future_to_tienda[future]
            try:
                status, data = future.result()
                if status == 'success':
                    lista_df_cierres_caja.append(data)
                elif status == 'error':
                    lista_errores_especificos_api.append(data)
                # 'no_data' no hace nada
            except Exception as exc:
                tienda_name = tienda.get('name', 'N/A')
                print(f"!! Excepción al procesar el futuro de la tienda {tienda_name}: {exc}")

else:
    print("No se puede proceder a obtener cierres de caja: df_licencias está vacío o faltan columnas 'id'/'token'.")


if lista_df_cierres_caja:
    print(f"\nSe obtuvieron datos de cierre de caja para {len(lista_df_cierres_caja)} tienda(s).")
    try:
        df_cierres_consolidado = pd.concat(lista_df_cierres_caja, ignore_index=True)
        print("\n--- Resumen del DataFrame Consolidado de Cierres de Caja ---")
        print(df_cierres_consolidado.head())
        print(f"Dimensiones del Consolidado: {df_cierres_consolidado.shape}")
    except Exception as e:
        print(f"Error inesperado durante la concatenación de DataFrames de cierre: {e}")
else:
    print("\nNo se obtuvieron datos de cierre de caja para ninguna tienda (lista_df_cierres_caja está vacía).")

if lista_errores_especificos_api:
    df_errores_tiendas = pd.DataFrame(lista_errores_especificos_api)
    print("\n--- DataFrame de Errores Específicos de API por Tienda (error:1, Index out of bounds) ---")
    print(df_errores_tiendas)
else:
    print("\nNo se registraron errores específicos de API del tipo 'Index was outside the bounds of the array.'")

# --- Seleccionar Columnas Específicas del DataFrame Consolidado ---
df_para_exportar = pd.DataFrame() 
columnas_existentes = [] 

if df_cierres_consolidado is not None and not df_cierres_consolidado.empty:
    columnas_deseadas = ['date', 'user', 'paymentInstrument', 'cashier', 'symbols', 'store_id', 'store_name'] 
    columnas_existentes = [col for col in columnas_deseadas if col in df_cierres_consolidado.columns]
    
    if columnas_existentes:
        df_para_exportar = df_cierres_consolidado[columnas_existentes].copy() 
        print(f"\nDataFrame con columnas seleccionadas para exportar ({', '.join(columnas_existentes)}):")
        print(df_para_exportar.head())
        if len(columnas_existentes) < len(columnas_deseadas):
            columnas_faltantes = [col for col in columnas_deseadas if col not in columnas_existentes]
            print(f"\nAdvertencia: Columnas deseadas no encontradas y no incluidas: {', '.join(columnas_faltantes)}")
    else: 
        print("\nNinguna de las columnas deseadas se encontró en df_cierres_consolidado.")
elif df_cierres_consolidado is not None and df_cierres_consolidado.empty:
    print("\nEl DataFrame df_cierres_consolidado está vacío. No se pueden seleccionar columnas para exportar.")
else: 
    print("\nEl DataFrame df_cierres_consolidado no está disponible (es None). No se pueden seleccionar columnas.")


Obteniendo datos de cierre de caja para la fecha: 2025-05-02
-> Iniciando petición para tienda: [ BCO ] - BOCONÓ (ID: 00001)
-> Iniciando petición para tienda: [ TRU ] - VALERA SHOP (ID: 00002)
-> Iniciando petición para tienda: [ SMT ] - SABANA MENDOZA - TRUJILLO (ID: 00003)
-> Iniciando petición para tienda: [ PFO ] - PUNTO FIJO (ID: 00005)
-> Iniciando petición para tienda: [ AVM ] - AVENIDA MARIÑO (ID: 00006)
-> Iniciando petición para tienda: [ BZD ] - BAZAR 2 (ID: 00007)
-> Iniciando petición para tienda: [ BZU ] - BAZAR 1 (ID: 00008)
-> Iniciando petición para tienda: [ TED ] - LOS TEQUES DOS (ID: 00009)
-> Iniciando petición para tienda: [ GRA ] - GUARENAS (ID: 00010)
-> Iniciando petición para tienda: [ TBD ] - TEMBLADOR (ID: 00011)
-> Iniciando petición para tienda: [ MGT ] - MARGARITA SHOP (ID: 00012)
-> Iniciando petición para tienda: [ PTC ] - PUERTO CABELLO (ID: 00013)
-> Iniciando petición para tienda: [ YTG ] - YARITAGUA (ID: 00014)
-> Iniciando petición para tienda: [ 

C:\Users\geren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\geren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\geren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarnin

-> Iniciando petición para tienda: [ PSA ] - ACARIGUA (ID: 00026)
-> Iniciando petición para tienda: [ CUA ] - CUMANA SHOP (ID: 00027)
-> Iniciando petición para tienda: [ GTN ] - GATO NEGRO (ID: 00028)
-> Iniciando petición para tienda: [ BVC ] - BAZAR VALENCIA CANGURO (ID: 00031)
-> Iniciando petición para tienda: [ TCP ] - TUCUPITA (ID: 00032)
-> Iniciando petición para tienda: [ CAT ] - CATIA (ID: 00033)
-> Iniciando petición para tienda: [ ODA ] - CIUDAD OJEDA (ID: 00034)
-> Iniciando petición para tienda: [ TJL ] - TRUJILLO (ID: 00035)
-> Iniciando petición para tienda: [ PTT ] - PETARE 3 (ID: 00036)
-> Iniciando petición para tienda: [ SMZ ] - SIERRA MAESTRA (ID: 00037)
-> Iniciando petición para tienda: [ APU ] - APURE (ID: 00040)
-> Iniciando petición para tienda: [ MJN ] - MOJAN (ID: 00041)


C:\Users\geren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\geren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\geren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarnin

<- Error HTTP 404 para tienda 00025.
-> Iniciando petición para tienda: [ VIG ] - VIGIA (ID: 00042)


...

<- Datos recibidos para tienda 00220 (1 filas).
-> Iniciando petición para tienda: [ TPV ] - TERMINAL PLAZA VENEZUELA (ID: 00252)


C:\Users\geren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<- Excepción de red para tienda 00201: HTTPSConnectionPool(host='127.0.0.1', port=5000): Read timed out. (read timeout=30)
-> Iniciando petición para tienda: [ MOC ] - CANGURO MOVIL (ID: 00253)


C:\Users\geren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<- Datos recibidos para tienda 00211 (1 filas).
<- Datos recibidos para tienda 00217 (1 filas).
<- Error HTTP 404 para tienda 00239.
<- Error HTTP 404 para tienda 00238.
<- Error HTTP 404 para tienda 00235.
<- Datos recibidos para tienda 00222 (1 filas).
<- Datos recibidos para tienda 00221 (1 filas).
<- Error HTTP 404 para tienda 00241.
<- Datos recibidos para tienda 00223 (1 filas).
<- Error HTTP 404 para tienda 00246.
<- Datos recibidos para tienda 00229 (1 filas).
<- Datos recibidos para tienda 00230 (1 filas).
<- Error HTTP 404 para tienda 00245.
<- Error HTTP 404 para tienda 00248.
<- Error HTTP 404 para tienda 00252.
<- Datos recibidos para tienda 00231 (1 filas).
<- Error HTTP 404 para tienda 00249.
<- Error HTTP 404 para tienda 00250.
<- Error HTTP 404 para tienda 00251.
<- Excepción de red para tienda 00219: HTTPSConnectionPool(host='127.0.0.1', port=5000): Read timed out. (read timeout=30)
<- Error HTTP 404 para tienda 00242.
<- Error HTTP 404 para tienda 00253.
<- Error HTT

In [21]:
# --- Exportar el DataFrame final a CSV ---
output_directory = '.' 
filename = "lista_df_cierres_caja.csv"
nombre_archivo_csv = os.path.join(output_directory, filename)

try:
    if df_para_exportar is not None and not df_para_exportar.empty:
        df_para_exportar.to_csv(nombre_archivo_csv, index=False, encoding='utf-8-sig')
        print(f"\nDataFrame final exportado exitosamente a '{nombre_archivo_csv}'")
    elif df_para_exportar is not None and df_para_exportar.empty and \
         (df_cierres_consolidado is not None and not df_cierres_consolidado.empty and not columnas_existentes):
         print("\nEl DataFrame final para exportar está vacío porque no se encontraron las columnas deseadas en los datos consolidados. No se generará CSV.")
    elif df_para_exportar is not None and df_para_exportar.empty:
         print("\nEl DataFrame final para exportar está vacío. No se generará el archivo CSV.")
    else: 
        print("\nEl DataFrame final para exportar no está disponible (es None). No se generará el archivo CSV.")
except Exception as e:
    print(f"\nOcurrió un error durante la exportación del DataFrame a CSV: {e}")


DataFrame final exportado exitosamente a '.\lista_df_cierres_caja.csv'


In [22]:
# --- NUEVA SECCIÓN: Procesar 'paymentInstrument' y Calcular Suma de Diferencias ---
print("\n--- Iniciando procesamiento de 'paymentInstrument' para calcular sumas y diferencias ---")

def procesar_payment_instrument_para_fila(payment_instrument_data, row_identifier_for_print):
    """
    Procesa la columna 'paymentInstrument' de una fila.
    Suma 'difference', 'saintAmount', y 'auditedAmount' divididos por 'factor'.
    Realiza prints de auditoría. 
    Ignora instrumentos con idPayment == "BASECASH".
    Retorna (total_difference_ajustada, total_saint_amount_ajustado, total_audited_amount_ajustado)
    """
    if not isinstance(payment_instrument_data, (list, str)):
        print(f"  Fila {row_identifier_for_print}: 'paymentInstrument' no es una lista ni un string JSON válido. Se omite. Tipo: {type(payment_instrument_data)}")
        return 0.0, 0.0, 0.0

    instrument_list = []
    if isinstance(payment_instrument_data, str):
        try:
            instrument_list = json.loads(payment_instrument_data)
            if not isinstance(instrument_list, list):
                print(f"  Fila {row_identifier_for_print}: 'paymentInstrument' (string decodificado) no es una lista. Se omite. Contenido: {str(instrument_list)[:100]}")
                return 0.0, 0.0, 0.0
        except json.JSONDecodeError:
            print(f"  Fila {row_identifier_for_print}: Error al decodificar JSON en 'paymentInstrument'. Se omite. Contenido: {str(payment_instrument_data)[:100]}")
            return 0.0, 0.0, 0.0
    elif isinstance(payment_instrument_data, list):
        instrument_list = payment_instrument_data 
    else: 
        print(f"  Fila {row_identifier_for_print}: Formato inesperado para 'paymentInstrument'. Se omite. Tipo: {type(payment_instrument_data)}")
        return 0.0, 0.0, 0.0

    total_difference_ajustada = 0.0
    total_saint_amount_ajustado = 0.0
    total_audited_amount_ajustado = 0.0
    
    print(f"  Auditoría para Fila {row_identifier_for_print}:")
    
    if not instrument_list: 
        print(f"    'paymentInstrument' está vacío o no contiene elementos procesables.")
        return 0.0, 0.0, 0.0

    for i, payment in enumerate(instrument_list):
        if isinstance(payment, dict):
            payment_id = payment.get('idPayment')
            payment_name = payment.get('name', 'N/A')

            if payment_id == "BASECASH":
                print(f"    - Instrumento '{payment_name}' (ID: {payment_id}) omitido por ser BASECASH.")
                continue 

            factor_value = None
            if 'factor' in payment:
                try:
                    factor_value = float(payment['factor'])
                    if factor_value == 0:
                        print(f"    - Instrumento '{payment_name}' (ID: {payment_id}): 'factor' es 0. No se pueden realizar divisiones para este instrumento.")
                        factor_value = None
                except (ValueError, TypeError):
                    print(f"    - Instrumento '{payment_name}' (ID: {payment_id}): 'factor' ('{payment['factor']}') no es numérico. Se omiten cálculos dependientes del factor.")
                    factor_value = None
            else:
                print(f"    - Instrumento '{payment_name}' (ID: {payment_id}): no tiene clave 'factor'. Se omiten cálculos dependientes del factor.")

            if 'difference' in payment and factor_value is not None:
                try:
                    difference_value = float(payment['difference'])
                    diferencia_ajustada = difference_value / factor_value
                    print(f"    - Instrumento '{payment_name}' (ID: {payment_id}): diferencia_orig = {difference_value}, factor = {factor_value}, diferencia_calc = {diferencia_ajustada:.4f}")
                    total_difference_ajustada += diferencia_ajustada
                except (ValueError, TypeError) as e:
                    print(f"    - Instrumento '{payment_name}' (ID: {payment_id}): valor 'difference' ('{payment['difference']}') no es numérico. Se omite cálculo. Error: {e}")
            elif 'difference' in payment and factor_value is None:
                print(f"    - Instrumento '{payment_name}' (ID: {payment_id}): tiene 'difference' pero 'factor' no es válido/disponible. No se calcula.")
            
            if 'saintAmount' in payment and factor_value is not None:
                try:
                    saint_amount_value = float(payment['saintAmount'])
                    saint_amount_ajustado = saint_amount_value / factor_value
                    print(f"    - Instrumento '{payment_name}' (ID: {payment_id}): saintAmount_orig = {saint_amount_value}, factor = {factor_value}, saintAmount_calc = {saint_amount_ajustado:.4f}")
                    total_saint_amount_ajustado += saint_amount_ajustado
                except (ValueError, TypeError) as e:
                    print(f"    - Instrumento '{payment_name}' (ID: {payment_id}): valor 'saintAmount' ('{payment['saintAmount']}') no es numérico. Se omite cálculo. Error: {e}")
            elif 'saintAmount' in payment and factor_value is None:
                print(f"    - Instrumento '{payment_name}' (ID: {payment_id}): tiene 'saintAmount' pero 'factor' no es válido/disponible. No se calcula.")
            
            if 'auditedAmount' in payment and factor_value is not None:
                try:
                    audited_amount_value = float(payment['auditedAmount'])
                    audited_amount_ajustado = audited_amount_value / factor_value
                    print(f"    - Instrumento '{payment_name}' (ID: {payment_id}): auditedAmount_orig = {audited_amount_value}, factor = {factor_value}, auditedAmount_calc = {audited_amount_ajustado:.4f}")
                    total_audited_amount_ajustado += audited_amount_ajustado
                except (ValueError, TypeError) as e:
                    print(f"    - Instrumento '{payment_name}' (ID: {payment_id}): valor 'auditedAmount' ('{payment['auditedAmount']}') no es numérico. Se omite cálculo. Error: {e}")
            elif 'auditedAmount' in payment and factor_value is None:
                 print(f"    - Instrumento '{payment_name}' (ID: {payment_id}): tiene 'auditedAmount' pero 'factor' no es válido/disponible. No se calcula.")
        else:
            print(f"    - Elemento {i+1} en 'paymentInstrument' no es un diccionario válido. Detalle: {str(payment)[:70]}")
            
    print(f"  Totales para Fila {row_identifier_for_print}: Suma Diferencias Ajustadas = {total_difference_ajustada:.4f}, Suma SaintAmount Ajustado = {total_saint_amount_ajustado:.4f}, Suma AuditedAmount Ajustado = {total_audited_amount_ajustado:.4f}")
    return total_difference_ajustada, total_saint_amount_ajustado, total_audited_amount_ajustado

df_con_diferencias = pd.DataFrame() 

if df_para_exportar is not None and not df_para_exportar.empty:
    df_con_diferencias = df_para_exportar.copy()
    
    if 'paymentInstrument' in df_con_diferencias.columns:
        print("\nProcesando filas para las nuevas columnas calculadas:")
        
        columna_diferencias_calculadas = []
        columna_saint_amount_total_ajustado = []
        columna_audited_amount_total_ajustado = []

        for index, row in df_con_diferencias.iterrows():
            row_id_parts = []
            if 'store_name' in row and pd.notna(row['store_name']): row_id_parts.append(f"Tienda: {row['store_name']}")
            if 'date' in row and pd.notna(row['date']): row_id_parts.append(f"Fecha: {row['date']}")
            row_id_parts.append(f"ÍndiceDF: {index}")
            row_identifier = ", ".join(row_id_parts)
            
            diff_adj, saint_total_adj, audited_total_adj = procesar_payment_instrument_para_fila(row['paymentInstrument'], row_identifier)
            
            columna_diferencias_calculadas.append(diff_adj)
            columna_saint_amount_total_ajustado.append(saint_total_adj)
            columna_audited_amount_total_ajustado.append(audited_total_adj)
            print("-" * 20) 

        df_con_diferencias['Diferencias'] = columna_diferencias_calculadas
        df_con_diferencias['TotalSaintAmountAjustado'] = columna_saint_amount_total_ajustado
        df_con_diferencias['TotalAuditedAmountAjustado'] = columna_audited_amount_total_ajustado
        
        columnas_a_eliminar = ['paymentInstrument', 'cashier', 'symbols', 'store_id']
        columnas_encontradas_para_eliminar = [col for col in columnas_a_eliminar if col in df_con_diferencias.columns]
        
        if columnas_encontradas_para_eliminar:
            df_con_diferencias = df_con_diferencias.drop(columns=columnas_encontradas_para_eliminar)
            print(f"\nColumnas {columnas_encontradas_para_eliminar} eliminadas de 'df_con_diferencias'.")

        print("\n--- DataFrame con nuevas columnas calculadas (primeras filas) ---")
        columnas_a_mostrar = ['date', 'user', 'store_name', 'Diferencias', 'TotalSaintAmountAjustado', 'TotalAuditedAmountAjustado']
        columnas_presentes_para_mostrar = [col for col in columnas_a_mostrar if col in df_con_diferencias.columns]
        
        if columnas_presentes_para_mostrar:
             print(df_con_diferencias[columnas_presentes_para_mostrar].head())
        elif not df_con_diferencias.empty: 
            print(df_con_diferencias.head())
        
    else:
        print("\nLa columna 'paymentInstrument' no se encontró en 'df_para_exportar'. No se pudieron calcular las nuevas columnas.")
else:
    print("\nEl DataFrame 'df_para_exportar' está vacío o no disponible. No se creará el DataFrame con las nuevas columnas calculadas.")




--- Iniciando procesamiento de 'paymentInstrument' para calcular sumas y diferencias ---

Procesando filas para las nuevas columnas calculadas:
  Auditoría para Fila Tienda: [ TBD ] - TEMBLADOR, Fecha: 2025-05-02T00:00:00, ÍndiceDF: 0:
    - Instrumento 'BASE DE CAJA (VES Bs)' (ID: BASECASH) omitido por ser BASECASH.
    - Instrumento 'EFECTIVO (VES Bs)' (ID: TACASH): diferencia_orig = 1395.2862, factor = 87.57, diferencia_calc = 15.9334
    - Instrumento 'EFECTIVO (VES Bs)' (ID: TACASH): saintAmount_orig = 2714.7138, factor = 87.57, saintAmount_calc = 31.0005
    - Instrumento 'EFECTIVO (VES Bs)' (ID: TACASH): auditedAmount_orig = 4110.0, factor = 87.57, auditedAmount_calc = 46.9339
    - Instrumento 'BASE DE CAJA (COP $)' (ID: BASECASH) omitido por ser BASECASH.
    - Instrumento 'EFECTIVO (COP $)' (ID: TACASH): diferencia_orig = 0.0, factor = 3950.0, diferencia_calc = 0.0000
    - Instrumento 'EFECTIVO (COP $)' (ID: TACASH): saintAmount_orig = 0.0, factor = 3950.0, saintAmount_calc

In [23]:
# --- Exportar el DataFrame final a CSV ---
output_directory = '.' # Directorio actual por defecto
filename = "lista_df_cierres_caja_Dif_General.csv"
nombre_archivo_csv = os.path.join(output_directory, filename)
print(df_con_diferencias.head)
try:
    if df_con_diferencias is not None and not df_con_diferencias.empty:
        df_con_diferencias.to_csv(nombre_archivo_csv, index=False, encoding='utf-8-sig')
        print(f"\nDataFrame final exportado exitosamente a '{nombre_archivo_csv}'")
    # Caso donde df_con_diferencias está vacío porque, aunque df_cierres_consolidado tenía datos, no se encontraron las columnas deseadas.
    elif df_con_diferencias is not None and df_con_diferencias.empty and \
         (df_cierres_consolidado is not None and not df_cierres_consolidado.empty and not columnas_existentes):
         print("\nEl DataFrame final para exportar está vacío porque no se encontraron las columnas deseadas en los datos consolidados. No se generará CSV.")
    # Caso general donde df_con_diferencias está vacío (p.ej., df_cierres_consolidado estaba vacío desde el inicio)
    elif df_con_diferencias is not None and df_con_diferencias.empty:
         print("\nEl DataFrame final para exportar está vacío. No se generará el archivo CSV.")
    else: # df_con_diferencias es None (no debería ocurrir con la inicialización)
        print("\nEl DataFrame final para exportar no está disponible (es None). No se generará el archivo CSV.")
except Exception as e:
    print(f"\nOcurrió un error durante la exportación del DataFrame a CSV: {e}")

<bound method NDFrame.head of                     date  user  \
0    2025-05-02T00:00:00  011V   
1    2025-05-02T00:00:00  001V   
2    2025-05-02T00:00:00  003V   
3    2025-05-02T00:00:00  020V   
4    2025-05-02T00:00:00  016V   
..                   ...   ...   
144  2025-05-02T00:00:00  240V   
145  2025-05-02T00:00:00  243v   
146  2025-05-02T00:00:00  244v   
147  2025-05-02T00:00:00  237V   
148  0001-01-01T00:00:00  None   

                                          store_name  Diferencias  \
0                                [ TBD ] - TEMBLADOR   122.946857   
1                                   [ BCO ] - BOCONÓ   222.933085   
2                [ SMT ] - SABANA MENDOZA - TRUJILLO     0.053044   
3                                 [ CHI ] - CHIVACOA     3.208837   
4                                    [ UPT ] - UPATA   223.361060   
..                                               ...          ...   
144                       [ CLP ] - CATIA LIFE PHONE     2.053897   
145      